In [ ]:
import os
import importlib
import time

import numpy as np
import pandas as pd

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import skimage
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler

import load_data
import model
#import first_exp_model_with_pytorch as dloader
#import first_exp_model_with_pytorch

In [ ]:
plt.style.use('Solarize_Light2')
%matplotlib inline
importlib.reload(load_data)
importlib.reload(model)

In [ ]:
W = H = 256

BATCH_SIZE = 100

LABEL_NAMES_MAP = {
    0:  "Nucleoplasm",  
    1:  "Nuclear membrane",   
    2:  "Nucleoli",   
    3:  "Nucleoli fibrillar center",   
    4:  "Nuclear speckles",
    5:  "Nuclear bodies",   
    6:  "Endoplasmic reticulum",   
    7:  "Golgi apparatus",   
    8:  "Peroxisomes",   
    9:  "Endosomes",   
    10:  "Lysosomes",   
    11:  "Intermediate filaments",   
    12:  "Actin filaments",   
    13:  "Focal adhesion sites",   
    14:  "Microtubules",   
    15:  "Microtubule ends",   
    16:  "Cytokinetic bridge",   
    17:  "Mitotic spindle",   
    18:  "Microtubule organizing center",   
    19:  "Centrosome",   
    20:  "Lipid droplets",   
    21:  "Plasma membrane",   
    22:  "Cell junctions",   
    23:  "Mitochondria",   
    24:  "Aggresome",   
    25:  "Cytosol",   
    26:  "Cytoplasmic bodies",   
    27:  "Rods & rings"
}

In [ ]:
def show(image, labels):
    f, (ax1,ax2,ax3,ax4) = plt.subplots(1, 4, figsize=(25,25), sharey=True)

    title = ''
    #npimage = image.numpy()[:, ::-1, :, :]
    #print(image.size())
    #print(labels)
    #print(labels.size())
    #print(image)
    npimage = image.numpy()
    nplabels = labels.numpy()
    print(npimage.shape)
    print(nplabels)
    #transposed_npimage = np.transpose(npimage, (1, 2, 0))
    
    #print(transposed_npimage.shape)
    
    #labels =sample['target']
                
    #for i, label in enumerate(LABELS):
    #    if labels[i] == 1:
    #        if title == '':
    #            title += label
    #        else:
    #            title += " & " + label
    
    #for i, label in enumerate(nplabels[0]):
    for i, label in enumerate(nplabels):
        if label == 1:
            if title == '':
                title += LABEL_NAMES_MAP[i]
            else:
                title += " & " + LABEL_NAMES_MAP[i]
    
    ax1.imshow(npimage[0,:,:],cmap="hot")
    ax1.set_title('Red')
    ax2.imshow(npimage[1,:,:],cmap="copper")
    ax2.set_title('Green')
    ax3.imshow(npimage[2,:,:],cmap="bone")
    ax3.set_title('Blue')
    ax4.imshow(npimage[3,:,:],cmap="afmhot")
    ax4.set_title('Yellow')
    f.suptitle(title, fontsize=20, y=0.62)
    

In [ ]:
class HPA_ConvNeuralNet(nn.Module):
    
    def __init__(self):
        super(HPA_ConvNeuralNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(4, 8, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.drop_out = nn.Dropout()
        self.out1 = nn.Linear(int(16 * W/4 * H/4), 900)
        self.out2 = nn.Linear(900, 28)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.drop_out(x)
        output = self.out1(x)
        output = self.out2(output)
        #return output, x
        return output

In [ ]:
def prepare_loaders(dataset, valid_train_ratio=0.6):
    dataset_size = len(dataset)
    print("dataset_size: ", dataset_size)
    #train_subset_size = int(valid_train_ratio * dataset_size)
    validation_subset_size = int(dataset_size * (1 - valid_train_ratio))
    print("validation_subset_size: ", validation_subset_size)

    indices = list(range(dataset_size))
    validation_indices = np.random.choice(indices, size=validation_subset_size, replace=False)
    train_indices = list(set(indices) - set(validation_indices))

    train_sampler = SubsetRandomSampler(train_indices)
    validation_sampler = SubsetRandomSampler(validation_indices)
    
    dataset_sizes = {
            'train': len(train_indices),
            'validation': len(validation_indices)
        }

    train_loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=1, sampler=train_sampler)
    validation_loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=1, sampler=validation_sampler)

    loaders = {
            'train': train_loader,
            'validation': validation_loader
        }

    return loaders, dataset_sizes

In [ ]:
def validate(model, validation_loader, validation_dataset_size):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        i = 0
        for images, labels in validation_loader:
            images = images.to(device, dtype=torch.float)
            print("images.size(): ", images.size())
            
            labels = labels.to(device, dtype=torch.uint8)
            
            print("labels.size(): ", labels.size())
            print("labels:\n", labels)
            print("labels.dtype: ", labels.dtype)
            outputs = model(images)
            #outputs = model(images)[0]
            print("outputs.size(): ", outputs.size())
            print("outputs:\n", outputs)
            print("outputs.data:\n", outputs.data)
            #_, predicted = torch.max(outputs.data, 1)
            #pred, predicted = torch.max(outputs.data, 1)
            predicted = outputs > 0
            #predicted = outputs.to(device, dtype=torch.int64)
            predicted = outputs.to(device, dtype=torch.uint8)
            print("predicted.size(): ", predicted.size())
            print("predicted:\n", predicted)
            #print("predicted.size(): ", predicted.size())
            #print("pred:\n", pred)
            total += labels.size(0)
            print("labels.sum(): ", labels.sum())
            print("predicted.sum():", predicted.sum())
            print("predicted == labels:\n", predicted == labels)
            correct += torch.sum((predicted == labels).all(1))
            i += 1
            if i == 10:
                break
        accuracy = correct / total
        print(
                "Validation accuracy of the model on the {} validation images: {} ".format(
                    validation_dataset_size,
                    accuracy
                )
            )

    return accuracy

In [ ]:
train_df = load_data.load_text_data('../input/train.csv')

In [ ]:
print(train_df.shape)

In [ ]:
dataset = load_data.HumanProteinAtlasDataset(
    train_df,
    transform=load_data.transforms.ToTensor()
)

In [ ]:
dataset_loaders, dataset_sizes = prepare_loaders(dataset)

In [ ]:
#train_loader = dloader.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)
#train_loader = load_data.data.DataLoader(train_dataset, batch_size=40, shuffle=True, num_workers=1)
train_loader = dataset_loaders['train']
validation_loader = dataset_loaders['validation']
train_dataset_size = dataset_sizes['train']
validation_dataset_size = dataset_sizes['validation']

In [ ]:
print(train_dataset_size)
print(validation_dataset_size)

In [ ]:
print(train_loader)

In [ ]:
train_iter = iter(train_loader)

In [ ]:
print(type(train_iter))

In [ ]:
images, labels = train_iter.next()

In [ ]:
print("images shape on batch size = {}".format(images.size()))
print("labels shape on batch size = {}".format(labels.size()))

In [ ]:
#import torchvision

In [ ]:
#grid = torchvision.utils.make_grid(images)
#grid = torchvision.utils.make_grid([image[0] for image in images])

In [ ]:
#plt.imshow(grid)
#plt.axis('off')
#plt.title(labels.numpy())

In [ ]:
show(images[0], labels[0])

In [ ]:
hpa_conv_neural_net = HPA_ConvNeuralNet()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

In [ ]:
hpa_conv_neural_net.to(device)

In [ ]:
#criterion = nn.CrossEntropyLoss()
criterion=nn.BCEWithLogitsLoss(reduction='sum')
#optimizer = optim.SGD(hpa_conv_neural_net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(hpa_conv_neural_net.parameters(), lr=0.001, betas=(0.9, 0.99))

In [ ]:
torch.cuda.empty_cache()
since = time.time()
for epoch in range(2):
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data
        inputs, labels = inputs.to(device, dtype=torch.float), labels.to(device, dtype=torch.float)
        print("inputs.size(): ", inputs.size())
        print("inputs:\n", inputs)
        #labels = labels[0]
        print("labels.size(): ", labels.size())
        print("labels: ", labels)
        optimizer.zero_grad()
        
        outputs = hpa_conv_neural_net(inputs)
        print("type(outputs): ", type(outputs))
        print("outputs.size(): ", outputs.size())
        print("outputs:\n", outputs)
        #loss = criterion(outputs, labels.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 200 == 199:
            print("[%d, %5d, loss: %.3f]" % (epoch + 1, i + 1, running_loss / 20.0))
            running_loss = 0.0
            
print("Finished Training")
time_elapsed = time.time() - since
print(
    'Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60
    )
)

In [ ]:
validation_accuracy = validate(hpa_conv_neural_net, validation_loader, validation_dataset_size)